# OpenDart (DART: Data Analysis, Retrieval and Transfer System, 전자공시시스템)

OpenDart 공시 문서 크롤러 모듈.

DART(전자공시시스템)에서 기업 공시 문서를 크롤링하여
GCS(Google Cloud Storage)에 업로드하는 기능을 제공합니다.

```
OpenDartCrawler
├── DartDocumentViewer (DART 문서 뷰어 클래스)
│   ├── fetch (공시 정보로부터 문서 정보 가져오기)
│   └── ...
├── DartDocumentParser (DART 문서 파싱 클래스)
│   ├── fetch (공시 번호로부터 첨부파일 목록을 가져오기)
│   ├── download (현재 종목의 시가총액을 스크래핑하여 숫자로 반환)
│   └── parse (...)
├── DartAPIParser (OpenDART API 파싱 클래스)
│   ├── list (공시검색, 공시 유형별, 회사별, 날짜별 등 여러가지 조건으로 공시보고서 검색기능을 제공합니다.)
│   ├── company (기업개황, 전반적인 상황, DART에 등록되어있는 기업의 개황정보를 제공합니다.)
│   ├── document (공시서류원본파일, 공시보고서 원본파일을 제공합니다.)
│   ├── corp_code (고유번호, DART에 등록되어있는 공시대상회사의 고유번호,회사명,종목코드, 최근변경일자를 파일로 제공합니다.)  
├── ...
├── check_gcp (GCP에서 Caching 정보 확인)
└── save_gcp (GCP에서 Caching 정보 저장)
```

- [공시정보 (Public Disclosure)](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001)
- [정기보고서 주요정보 (Key Information in Periodic Reports)](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS002)
- [정기보고서 재무정보 (Financial Information in Periodic Reports)](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS003)
- [지분공시 종합정보 (Comprehensive Share Ownership Information)](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS004)
- [주요사항보고서 주요정보 (Key Information in Reports on Material Facts)](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS005)
- [증권신고서 주요정보 (Key Information in Registration Statements)](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS006)

In [1]:
import io
import json
import os
import OpenDartReader
import pandas as pd
import re
import requests
import time
import zipfile

from contextlib import contextmanager
from dataclasses import dataclass
from datetime import datetime
from typing import Iterator, Dict, Any, List, Tuple, Optional
from bs4 import BeautifulSoup, Tag

#### Jupyter Notebook 설정

Jupyter Notebook에서 필요한 라이브러리 로딩을 위한 설정

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from utils import gcpmanager
from utils.companydict import companydict

## OpenDartReader 테스트

In [ ]:
!export DART_API_KEY=fd664865257f1a3073b654f9185de11a708f726c

#### OpenDart 등록 목록 조회 (OpenDartReader)

In [ ]:
open_dart = OpenDartReader("fd664865257f1a3073b654f9185de11a708f726c")
#df = open_dart.list("005930", start="2025-12-01", end="2025-12-31")
df = open_dart.list("005930", start="2025-10-01", end="2025-10-31")
df

#### 공시검색

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/120.0.0.0 Safari/537.36'
}

"""
{'status': '100', 'message': 'corp_code가 없는 경우 검색기간은 3개월만 가능합니다.'}
"""

url = "https://opendart.fss.or.kr/api/list.json"
params = {
    "crtfc_key": "fd664865257f1a3073b654f9185de11a708f726c",
    "corp_code": "005930",
    "bgn_de": "20250101",
    "end_de": "20251231",
    "corp_cls": "Y",
    "page_no": 1,
    "page_count": 100
}
response = requests.get(url, params=params, headers=headers)
response.encoding = 'utf-8'
json_text = response.json()
json_text

https://opendart.fss.or.kr/api/list.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=005930&bgn_de=20251001&end_de=20251031&corp_cls=Y&page_no=1&page_count=100

```json
{'status': '000','message': '정상','page_no': 1,'page_count': 100,'total_count': 19,'total_page': 1,
'list': [
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '주식등의대량보유상황보고서(일반)','rcept_no': '20251031000575','flr_nm': '삼성물산','rcept_dt': '20251031','rm': ''},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '주요사항보고서(자기주식처분결정)','rcept_no': '20251030000502','flr_nm': '삼성전자','rcept_dt': '20251030','rm': ''},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '특수관계인에대한증여','rcept_no': '20251030000495','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '공'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '특수관계인에대한증여','rcept_no': '20251030000492','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '공'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '기타경영사항(자율공시)              ','rcept_no': '20251030800673','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '기타경영사항(자율공시)              ','rcept_no': '20251030800671','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '기타경영사항(자율공시)              ','rcept_no': '20251030800649','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '장래사업ㆍ경영계획(공정공시)              ','rcept_no': '20251030800076','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '현금ㆍ현물배당결정              ','rcept_no': '20251030800075','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '[기재정정]연결재무제표기준영업(잠정)실적(공정공시)              ','rcept_no': '20251030800065','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '연결재무제표기준영업(잠정)실적(공정공시)              ','rcept_no': '20251030800056','flr_nm': '삼성전자','rcept_dt': '20251030','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '주식등의대량보유상황보고서(일반)','rcept_no': '20251024000542','flr_nm': '삼성물산','rcept_dt': '20251024','rm': ''},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '임원ㆍ주요주주특정증권등소유상황보고서','rcept_no': '20251021000291','flr_nm': '최철환','rcept_dt': '20251021','rm': ''},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '주식등의대량보유상황보고서(일반)',
'rcept_no': '20251017000512','flr_nm': '삼성물산','rcept_dt': '20251017','rm': ''},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '기업설명회(IR)개최(안내공시)              ','rcept_no': '20251014800035','flr_nm': '삼성전자','rcept_dt': '20251014','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '연결재무제표기준영업(잠정)실적(공정공시)              ','rcept_no': '20251014800003','flr_nm': '삼성전자','rcept_dt': '20251014','rm': '유정'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '최대주주등소유주식변동신고서              ','rcept_no': '20251013800563','flr_nm': '삼성전자','rcept_dt': '20251013','rm': '유'},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '자기주식취득결과보고서','rcept_no': '20251002000524','flr_nm': '삼성전자','rcept_dt': '20251002','rm': ''},
{'corp_code': '00126380','corp_name': '삼성전자','stock_code': '005930','corp_cls': 'Y','report_nm': '임원ㆍ주요주주특정증권등소유상황보고서','rcept_no': '20251001000542','flr_nm': '박승일','rcept_dt': '20251001','rm': ''}]}
```

```
    corp_code	corp_name	stock_code	corp_cls	report_nm	rcept_no	flr_nm	rcept_dt	rm
0	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251203000030	박성열	20251203	
1	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251203000019	백승엽	20251203	
2	00126380	삼성전자	005930	Y	[기재정정]동일인등출자계열회사와의상품ㆍ용역거래	20251203000011	삼성전자	20251203	공
3	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000167	민경일	20251202	
4	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000121	이충현	20251202	
5	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000089	김지웅	20251202	
6	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000045	이인실	20251202	
7	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000026	이성심	20251202	
8	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000017	고진일	20251202	
9	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000008	김철민	20251202	
10	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000007	장혁	20251202	
11	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000006	황현익	20251202	
12	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251202000002	이형주	20251202	
13	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000783	김경아	20251201	
14	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000708	정재훈	20251201	
15	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000615	최고은	20251201	
16	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000560	권주성	20251201	
17	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000525	권석남	20251201	
18	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000511	권주성	20251201	
19	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000457	전형석	20251201	
20	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000436	서무현	20251201	
21	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000278	우성훈	20251201	
22	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000265	문희철	20251201	
23	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000262	황정호	20251201	
24	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000216	박영재	20251201	
25	00126380	삼성전자	005930	Y	대규모기업집단현황공시[분기별공시(대표회사용)]	20251201000080	삼성전자	20251201	공
26	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000041	윤승국	20251201	
27	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000033	곽호석	20251201	
28	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000006	최성훈	20251201	
29	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251201000004	김원종	20251201	
30	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251128002192	문성수	20251201	
31	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251128002188	김운	20251201	
32	00126380	삼성전자	005930	Y	임원ㆍ주요주주특정증권등소유상황보고서	20251128002187	이종해	20251201	
```

#### OpenDart 첨부파일 조회 (OpenDartReader)

- http://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20251201000783&dcm_no=10906378
- https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20251201000783

In [ ]:
rcept_no = "20251201000783"
files = open_dart.attach_files(rcept_no)
files

#### Dart Configuration

In [ ]:
@dataclass(frozen=True)
class DartConfig:
    """DART 크롤러 설정."""
    
    bucket_name: str
    api_key: str
    
    @classmethod
    def from_env(cls) -> "DartConfig":
        """환경 변수에서 설정을 로드합니다."""
        bucket_name = "sayouzone-ai-stocks"
        #api_key = os.environ.get("DART_API_KEY")
        api_key = "fd664865257f1a3073b654f9185de11a708f726c"
        
        if not api_key:
            raise ValueError("DART_API_KEY 환경 변수가 설정되지 않았습니다.")
        
        return cls(bucket_name=bucket_name, api_key=api_key)

#### Utilities

In [ ]:
@contextmanager
def temporary_chdir(destination: str) -> Iterator[None]:
    """임시로 작업 디렉토리를 변경하는 컨텍스트 매니저."""
    original_dir = os.getcwd()
    try:
        os.chdir(destination)
        yield
    finally:
        os.chdir(original_dir)


def is_cloud_run_environment() -> bool:
    """Cloud Run 환경인지 확인합니다."""
    return bool(os.environ.get("K_SERVICE"))

## DART Attachment Parser

#### DartDocumentParser

DART 첨부파일 파싱을 담당하는 클래스

주요 파라미터
- rcp_no: 접수번호 (보고서 고유 ID) (rcept_no, rcpNo)
- dcm_no: 문서번호 (보고서 내의 하위 문서 ID) (dcmNo)
- ele_id: 엘리먼트 ID (요소 ID, 문서 목차의 특정 위치) (eleId)
- offset: 오프셋
- length: 길이
- dtd: DTD 파일명
- toc_no: (tocNo)
- atoc_id: (atocId)

개발 가이드
- [공시정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001)
- [공시정보 - 공시검색](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001)
- [공시정보 - 기업개황](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002)
- [공시정보 - 공시서류원본파일](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019003)
- [공시정보 - 고유번호](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018)
- [정기보고서 주요정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS002)
- [정기보고서 재무정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS003)
- [지분공시 종합정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS004)
- [주요사항보고서 주요정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS005)
- [증권신고서 주요정보](https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS006)

사용예시:<br/>
[공시검색](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001)<br/>
https://opendart.fss.or.kr/api/list.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&bgn_de=20200117&end_de=20200117&corp_cls=Y&page_no=1&page_count=10

```json
{
"status":"000",
"message":"정상",
"page_no":1,
"page_count":10,
"total_count":216,
"total_page":22,
"list":[
    {"corp_code":"00120182","corp_name":"NH투자증권","stock_code":"005940","corp_cls":"Y","report_nm":"[첨부추가]일괄신고추가서류(파생결합증권-주가연계증권)","rcept_no":"20200117000559","flr_nm":"NH투자증권","rcept_dt":"20200117","rm":""},
    {"corp_code":"00120182","corp_name":"NH투자증권","stock_code":"005940","corp_cls":"Y","report_nm":"[첨부추가]일괄신고추가서류(파생결합증권-주가연계증권)","rcept_no":"20200117000486","flr_nm":"NH투자증권","rcept_dt":"20200117","rm":""},
    {"corp_code":"00120182","corp_name":"NH투자증권","stock_code":"005940","corp_cls":"Y","report_nm":"[첨부추가]일괄신고추가서류(파생결합증권-주가연계증권)","rcept_no":"20200117000375","flr_nm":"NH투자증권","rcept_dt":"20200117","rm":""},
    {"corp_code":"00120182","corp_name":"NH투자증권","stock_code":"005940","corp_cls":"Y","report_nm":"[첨부추가]일괄신고추가서류(파생결합증권-주가연계증권)","rcept_no":"20200117000341","flr_nm":"NH투자증권","rcept_dt":"20200117","rm":""},
    {"corp_code":"00120182","corp_name":"NH투자증권","stock_code":"005940","corp_cls":"Y","report_nm":"[첨부추가]일괄신고추가서류(파생결합증권-주가연계증권)","rcept_no":"20200117000083","flr_nm":"NH투자증권","rcept_dt":"20200117","rm":""},
    {"corp_code":"00120182","corp_name":"NH투자증권","stock_code":"005940","corp_cls":"Y","report_nm":"[첨부추가]일괄신고추가서류(파생결합증권-주가연계증권)","rcept_no":"20200117000030","flr_nm":"NH투자증권","rcept_dt":"20200117","rm":""},
    {"corp_code":"00878915","corp_name":"iM금융지주","stock_code":"139130","corp_cls":"Y","report_nm":"소송등의판결ㆍ결정(자회사의 주요경영사항)","rcept_no":"20200117800593","flr_nm":"iM금융지주","rcept_dt":"20200117","rm":"유"},
    {"corp_code":"00120571","corp_name":"롯데칠성음료","stock_code":"005300","corp_cls":"Y","report_nm":"타법인주식및출자증권취득결정","rcept_no":"20200117800584","flr_nm":"롯데칠성음료","rcept_dt":"20200117","rm":"유정"},
    {"corp_code":"00161709","corp_name":"퍼시스","stock_code":"016800","corp_cls":"Y","report_nm":"주식등의대량보유상황보고서(약식)","rcept_no":"20200117000661","flr_nm":"피델리티매니지먼트앤리서치컴퍼니엘엘씨","rcept_dt":"20200117","rm":""},
    {"corp_code":"00188089","corp_name":"한섬","stock_code":"020000","corp_cls":"Y","report_nm":"주식등의대량보유상황보고서(약식)","rcept_no":"20200117000657","flr_nm":"피델리티매니지먼트앤리서치컴퍼니엘엘씨","rcept_dt":"20200117","rm":""}
]
}
```

[기업개황](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002)<br/>
https://opendart.fss.or.kr/api/company.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00126380

```json
{
"status":"000",
"message":"정상",
"corp_code":"00126380",
"corp_name":"삼성전자(주)",
"corp_name_eng":"SAMSUNG ELECTRONICS CO,.LTD",
"stock_name":"삼성전자",
"stock_code":"005930",
"ceo_nm":"전영현, 노태문",
"corp_cls":"Y",
"jurir_no":"1301110006246",
"bizr_no":"1248100998",
"adres":"경기도 수원시 영통구  삼성로 129 (매탄동)",
"hm_url":"www.samsung.com/sec",
"ir_url":"",
"phn_no":"02-2255-0114",
"fax_no":"031-200-7538",
"induty_code":"264",
"est_dt":"19690113",
"acc_mt":"12"
}
```

[단일회사 전체 재무제표](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019020)<br/>
https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00126380&bsns_year=2018&reprt_code=11011&fs_div=OFS

reprt_code: 보고서 코드
- 1분기보고서 : 11013
- 반기보고서 : 11012
- 3분기보고서 : 11014
- 사업보고서 : 11011

```json
{"status":"000","message":"정상","list":[{"rcept_no":"20190401004781","reprt_code":"11011","bsns_year":"2018","corp_code":"00126380","sj_div":"BS","sj_nm":"재무상태표","account_id":"ifrs_CurrentAssets","account_nm":"유동자산","account_detail":"-","thstrm_nm":"제 50 기","thstrm_amount":"80039455000000","frmtrm_nm":"제 49 기","frmtrm_amount":"70155189000000","bfefrmtrm_nm":"제 48 기","bfefrmtrm_amount":"69981128000000","ord":"1","currency":"KRW"},...,{"rcept_no":"20190401004781","reprt_code":"11011","bsns_year":"2018","corp_code":"00126380","sj_div":"SCE","sj_nm":"자본변동표","account_id":"ifrs_Equity","account_nm":"기말자본","account_detail":"자본 [member]|기타자본항목","thstrm_nm":"제 50 기","thstrm_amount":"1131186000000","frmtrm_nm":"제 49 기","frmtrm_amount":"-4660356000000","bfefrmtrm_nm":"제 48 기","bfefrmtrm_amount":"-8502219000000","ord":"12","currency":"KRW"}]}
```

[단일회사 주요 재무지표](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2022001)<br/>
https://opendart.fss.or.kr/api/fnlttSinglIndx.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00164742&bsns_year=2023&reprt_code=11014&idx_cl_code=M210000

reprt_code: 보고서 코드
- 1분기보고서 : 11013
- 반기보고서 : 11012
- 3분기보고서 : 11014
- 사업보고서 : 11011

```json
{"status":"000","message":"정상","list":[{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211100","idx_nm":"세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211200","idx_nm":"순이익률","idx_val":"7.327"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211250","idx_nm":"총포괄이익률","idx_val":"6.961"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211300","idx_nm":"매출총이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211400","idx_nm":"매출원가율"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211550","idx_nm":"ROE","idx_val":"7.816"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211800","idx_nm":"판관비율"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212000","idx_nm":"총자산영업이익률","idx_val":"4.76"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212100","idx_nm":"총자산세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212200","idx_nm":"자기자본영업이익률","idx_val":"11.874"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212300","idx_nm":"자기자본세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212400","idx_nm":"자본금영업이익률","idx_val":"4775.355"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212500","idx_nm":"자본금세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212600","idx_nm":"납입자본이익률","idx_val":"3143.328"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212700","idx_nm":"영업수익경비율"}]}
```

[다중회사 주요 재무지표](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2022002)<br/>
https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00164742,00159023&bsns_year=2023&reprt_code=11014&idx_cl_code=M210000

reprt_code: 보고서 코드
- 1분기보고서 : 11013
- 반기보고서 : 11012
- 3분기보고서 : 11014
- 사업보고서 : 11011

```json
{"status":"000","message":"정상","list":[{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211100","idx_nm":"세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211200","idx_nm":"순이익률","idx_val":"7.327"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211250","idx_nm":"총포괄이익률","idx_val":"6.961"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211300","idx_nm":"매출총이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211400","idx_nm":"매출원가율"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211550","idx_nm":"ROE","idx_val":"7.816"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211800","idx_nm":"판관비율"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212000","idx_nm":"총자산영업이익률","idx_val":"4.76"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212100","idx_nm":"총자산세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212200","idx_nm":"자기자본영업이익률","idx_val":"11.874"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212300","idx_nm":"자기자본세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212400","idx_nm":"자본금영업이익률","idx_val":"4775.355"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212500","idx_nm":"자본금세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212600","idx_nm":"납입자본이익률","idx_val":"3143.328"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00159023","stock_code":"017670","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212700","idx_nm":"영업수익경비율"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211100","idx_nm":"세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211200","idx_nm":"순이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211250","idx_nm":"총포괄이익률","idx_val":"9.793"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211300","idx_nm":"매출총이익률","idx_val":"20.719"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211400","idx_nm":"매출원가율","idx_val":"79.281"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211550","idx_nm":"ROE"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M211800","idx_nm":"판관비율","idx_val":"11.092"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212000","idx_nm":"총자산영업이익률","idx_val":"4.355"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212100","idx_nm":"총자산세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212200","idx_nm":"자기자본영업이익률","idx_val":"12.125"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212300","idx_nm":"자기자본세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212400","idx_nm":"자본금영업이익률","idx_val":"782.571"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212500","idx_nm":"자본금세전계속사업이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212600","idx_nm":"납입자본이익률"},{"reprt_code":"11014","bsns_year":"2023","corp_code":"00164742","stock_code":"005380","stlm_dt":"2023-09-30","idx_cl_code":"M210000","idx_cl_nm":"수익성지표","idx_code":"M212700","idx_nm":"영업수익경비율","idx_val":"11.092"}]}
```

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/120.0.0.0 Safari/537.36'
}

"""
{'status': '100', 'message': 'corp_code가 없는 경우 검색기간은 3개월만 가능합니다.'}
"""

url = "https://opendart.fss.or.kr/api/list.json"
params = {
    "crtfc_key": "fd664865257f1a3073b654f9185de11a708f726c",
    "bgn_de": "20251001",
    "end_de": "20251231",
    "corp_cls": "Y",
    "page_no": 1,
    "page_count": 100
}
response = requests.get(url, params=params, headers=headers)
response.encoding = 'utf-8'
json_text = response.json()
json_text

```json
{
'status': '000',
'message': '정상',
'page_no': 1,
'page_count': 10,
'total_count': 8671,
'total_page': 868,
'list': [
{'corp_code': '00113359',
'corp_name': '교보증권',
'stock_code': '030610',
'corp_cls': 'Y',
'report_nm': '증권발행실적보고서',
'rcept_no': '20251204000016',
'flr_nm': '교보증권',
'rcept_dt': '20251204',
'rm': ''},
{'corp_code': '00152862',
'corp_name': '코오롱',
'stock_code': '002020',
'corp_cls': 'Y',
'report_nm': '타인에대한채무보증결정(자회사의 주요경영사항)              ',
'rcept_no': '20251203800567',
'flr_nm': '코오롱',
'rcept_dt': '20251203',
'rm': '유'},
{'corp_code': '01363818',
'corp_name': '롯데리츠',
'stock_code': '330590',
'corp_cls': 'Y',
'report_nm': '[기재정정]부동산투자회사부동산처분              ',
'rcept_no': '20251203800574',
'flr_nm': '롯데리츠',
'rcept_dt': '20251203',
'rm': '유'},
{'corp_code': '00120562',
'corp_name': '롯데지주',
'stock_code': '004990',
'corp_cls': 'Y',
'report_nm': '임원ㆍ주요주주특정증권등소유상황보고서',
'rcept_no': '20251203000358',
'flr_nm': '윤현식',
'rcept_dt': '20251203',
'rm': ''},
{'corp_code': '00152880',
'corp_name': '코오롱글로벌',
'stock_code': '003070',
'corp_cls': 'Y',
'report_nm': '타인에대한채무보증결정              ',
'rcept_no': '20251203800566',
'flr_nm': '코오롱글로벌',
'rcept_dt': '20251203',
'rm': '유'},
{'corp_code': '00106119',
'corp_name': '금양',
'stock_code': '001570',
'corp_cls': 'Y',
'report_nm': '[기재정정]단기차입금증가결정(자율공시)              ',
'rcept_no': '20251203800556',
'flr_nm': '금양',
'rcept_dt': '20251203',
'rm': '유'},
{'corp_code': '00137997',
'corp_name': '현대차증권',
'stock_code': '001500',
'corp_cls': 'Y',
'report_nm': '투자설명서(일괄신고)',
'rcept_no': '20251203000350',
'flr_nm': '현대차증권',
'rcept_dt': '20251203',
'rm': ''},
{'corp_code': '00111421',
'corp_name': '휴니드테크놀러지스',
'stock_code': '005870',
'corp_cls': 'Y',
'report_nm': '[기재정정]주요사항보고서(자기주식처분결정)',
'rcept_no': '20251203000344',
'flr_nm': '휴니드테크놀러지스',
'rcept_dt': '20251203',
'rm': ''},
{'corp_code': '00111421',
'corp_name': '휴니드테크놀러지스',
'stock_code': '005870',
'corp_cls': 'Y',
'report_nm': '[기재정정]주요사항보고서(교환사채권발행결정)',
'rcept_no': '20251203000343',
'flr_nm': '휴니드테크놀러지스',
'rcept_dt': '20251203',
'rm': ''},
{'corp_code': '01535150',
'corp_name': 'SK리츠',
'stock_code': '395400',
'corp_cls': 'Y',
'report_nm': '특수관계인에대한주식의처분',
'rcept_no': '20251203000346',
'flr_nm': 'SK리츠',
'rcept_dt': '20251203',
'rm': '공'}
]
}
```

In [ ]:
class DartDocumentViewer:
    """DART 문서 뷰어 클래스"""
    
    main_url = "https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}"
    pdf_url = "https://dart.fss.or.kr/pdf/download/pdf.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
    pdf_main_url = "http://dart.fss.or.kr/pdf/download/main.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
    viewer_url = "https://dart.fss.or.kr/report/viewer.do"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/120.0.0.0 Safari/537.36'
    }
    
    @classmethod
    def fetch(self, content: dict) -> dict[str, str]:
        """공시 정보로부터 문서 정보를 가져옵니다.
        
        Args:
            content: 접수번호(rcept_no)
            
        Returns:
            파일명과 다운로드 URL의 딕셔너리
        """
        """
        {'text': '임원ㆍ주요주주 특정증권등 소유상황보고서', 'id': '1', 'rcpNo': '20251201000783', 'dcmNo': '10906378', 'eleId': '1', 'offset': '689', 'length': '2509', 'dtd': 'dart4.xsd', 'tocNo': '1', 'atocId': '1'}, 
        """
        result = self._parse_viewer(content)
        self._print_result(result)
        
        return result

    @classmethod
    def _parse_viewer(self, content: dict) -> dict[str, str]:
        """
        DART 공시 문서 내용 파싱
        
        Args:
            content (dict): Dictionary(
            rcp_no: 접수번호
            dcm_no: 문서번호
            ele_id: 요소 ID
            offset: 오프셋
            length: 길이
            dtd: DTD 파일명
            )
        
        Returns:
            dict: 파싱된 문서 내용
        """
        rcp_no = content.get("rcpNo")

        headers = self.headers
        headers['Referer'] = self.main_url.format(rcp_no=rcp_no)

        response = requests.get(self.viewer_url, params=content, headers=headers)
        response.encoding = 'utf-8'
        html = response.text

        soup = BeautifulSoup(html, 'html.parser')
    
        result = {
            'raw_html': html,
            'title': None,
            'tables': [],
            'text_content': None
        }
        
        # 제목 추출
        title_tag = soup.find('title')
        if title_tag:
            result['title'] = title_tag.get_text(strip=True)
        
        # 테이블 추출
        tables = soup.find_all('table')
        for idx, table in enumerate(tables):
            table_data = self._parse_viewer_table(table)
            if table_data:
                result['tables'].append({
                    'index': idx,
                    'data': table_data
                })
        
        # 텍스트 내용 추출
        body = soup.find('body')
        if body:
            result['text_content'] = body.get_text(separator='\n', strip=True)
        
        return result

    @classmethod
    def _parse_viewer_table(self, table):
        """HTML 테이블을 2D 리스트로 변환"""
        rows = []
        
        for tr in table.find_all('tr'):
            row = []
            for cell in tr.find_all(['td', 'th']):
                text = cell.get_text(separator=' ', strip=True)
                text = re.sub(r'\s+', ' ', text)  # 공백 정리
                row.append(text)
            
            if row:
                rows.append(row)
        
        return rows

    @classmethod
    def _print_result(self, result):
        """결과 출력"""
        print("=" * 70)
        print(f"제목: {result['title']}")
        print("=" * 70)
        
        print("\n[텍스트 내용]")
        print("-" * 70)
        if result['text_content']:
            # 처음 1000자만 출력
            content = result['text_content'][:1000]
            print(content)
            if len(result['text_content']) > 1000:
                print(f"\n... (총 {len(result['text_content'])}자)")
        
        print(f"\n[테이블 수: {len(result['tables'])}개]")
        print("-" * 70)
        
        for table in result['tables']:
            print(f"\n테이블 #{table['index'] + 1}")
            for row in table['data'][:5]:  # 처음 5행만
                print(row)
            if len(table['data']) > 5:
                print(f"... (총 {len(table['data'])}행)")

In [ ]:
class DartDocumentParser:
    """DART 문서 파싱 클래스"""
    
    main_url = "https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}"
    pdf_url = "https://dart.fss.or.kr/pdf/download/pdf.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
    pdf_main_url = "http://dart.fss.or.kr/pdf/download/main.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
    viewer_url = "https://dart.fss.or.kr/report/viewer.do"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/120.0.0.0 Safari/537.36'
    }
    
    def fetch(self, rcp_no: str) -> dict[str, str]:
        """공시 번호로부터 첨부파일 목록을 가져옵니다.
        
        Args:
            rcp_no: 접수번호(rcept_no)
            
        Returns:
            파일명과 다운로드 URL의 딕셔너리
        """
        main_page_url = self.main_url.format(rcp_no=rcp_no)
        print(f"OpenDart URL: {main_page_url}")
        
        response = requests.get(main_page_url, headers=self.headers, timeout=30)
        response.raise_for_status()

        toc = self._extract_toc(response.text)
        print(toc)
        """
        [
        {'text': '임원ㆍ주요주주 특정증권등 소유상황보고서', 'id': '1', 'rcpNo': '20251201000783', 'dcmNo': '10906378', 'eleId': '1', 'offset': '689', 'length': '2509', 'dtd': 'dart4.xsd', 'tocNo': '1', 'atocId': '1'}, 
        {'text': '1. 발행회사에 관한 사항', 'id': '2', 'rcpNo': '20251201000783', 'dcmNo': '10906378', 'eleId': '2', 'offset': '3477', 'length': '1495', 'dtd': 'dart4.xsd', 'tocNo': '2', 'atocId': '2'}, 
        {'text': '2. 보고자에 관한 사항', 'id': '3', 'rcpNo': '20251201000783', 'dcmNo': '10906378', 'eleId': '3', 'offset': '4976', 'length': '6149', 'dtd': 'dart4.xsd', 'tocNo': '3', 'atocId': '3'}, 
        {'text': '3. 특정증권등의 소유상황', 'id': '4', 'rcpNo': '20251201000783', 'dcmNo': '10906378', 'eleId': '4', 'offset': '11129', 'length': '14392', 'dtd': 'dart4.xsd', 'tocNo': '4', 'atocId': '4'}
        ]
        """

        viewer = DartDocumentViewer()
        for content in toc:
            result = viewer.fetch(content)

        pdf_info = self._extract_pdf_info(response.text)
        print(pdf_info)

        download_path = self._download_file(pdf_info)
        
        return self._parse_download_page(pdf_info)

    def _extract_toc(self, html_text: str):
        """
        DART 공시 페이지에서 목차(treeData) 추출
        
        Args:
            rcp_no: 접수번호 (예: "20251201000783")
        
        Returns:
            list: treeData 목록
        """
        
        # makeToc 함수 영역 추출
        toc_match = re.search(r'function makeToc\(\)\s*\{(.*?)\n\s*//js tree', html_text, re.DOTALL)
        
        if not toc_match:
            print("makeToc 함수를 찾을 수 없습니다.")
            return []
        
        toc_code = toc_match.group(1)
        
        # treeData 추출
        tree_data = []
        fields = ['text', 'id', 'rcpNo', 'dcmNo', 'eleId', 'offset', 'length', 'dtd', 'tocNo', 'atocId']
        
        # 각 node 블록 분리
        blocks = toc_code.split('treeData.push(node1);')
        
        for block in blocks[:-1]:
            node = {}
            for field in fields:
                # node1['field'] = "value"; 패턴 매칭
                pattern = rf"node1\['{field}'\]\s*=\s*\"([^\"]*)\""
                match = re.search(pattern, block)
                if match:
                    node[field] = match.group(1)
            
            if node:
                tree_data.append(node)
        
        return tree_data

    def _extract_pdf_info(self, html_text: str):
        # openPdfDownload('rcpNo', 'dcmNo') 패턴 찾기
        pattern = r"openPdfDownload\s*\(\s*['\"]([^'\"]+)['\"]\s*,\s*['\"]([^'\"]+)['\"]\s*\)"
        match = re.search(pattern, html_text)
        
        if match:
            result = {
                'rcpNo': match.group(1),
                'dcmNo': match.group(2),
                'pdf_url': self.pdf_url.format(rcp_no=match.group(1),dcm_no=match.group(2))
            }
            return result
        
        # 다른 패턴 시도: 변수로 전달되는 경우
        # openPdfDownload(rcpNo, dcmNo) 형태
        pattern2 = r"openPdfDownload\s*\(\s*(\w+)\s*,\s*(\w+)\s*\)"
        match2 = re.search(pattern2, html)
        
        if match2:
            # 변수 값 찾기
            var1_pattern = rf"var\s+{match2.group(1)}\s*=\s*['\"]([^'\"]+)['\"]"
            var2_pattern = rf"var\s+{match2.group(2)}\s*=\s*['\"]([^'\"]+)['\"]"
            
            var1_match = re.search(var1_pattern, html)
            var2_match = re.search(var2_pattern, html)
            
            if var1_match and var2_match:
                result = {
                    'rcpNo': var1_match.group(1),
                    'dcmNo': var2_match.group(1),
                    'pdf_url': self.pdf_url.format(rcp_no=var1_match.group(1),dcm_no=var2_match.group(1))
                }
                return result
        
        # rcpNo, dcmNo 직접 추출 시도
        rcp_pattern = r"rcpNo\s*[=:]\s*['\"]?(\d+)['\"]?"
        dcm_pattern = r"dcmNo\s*[=:]\s*['\"]?(\d+)['\"]?"
        
        rcp_match = re.search(rcp_pattern, html)
        dcm_match = re.search(dcm_pattern, html)
        
        if rcp_match and dcm_match:
            result = {
                'rcpNo': rcp_match.group(1),
                'dcmNo': dcm_match.group(1),
                'pdf_url': self.pdf_url.format(rcp_no=rcp_match.group(1),dcm_no=dcm_match.group(1))
            }
            return result
        
        return None

    def _download_file(self, pdf_info: dict, save_path: str = None):
        """
        DART 공시 PDF 다운로드
        
        Args:
            pdf_info: Dictionary (접수번호, 문서번호)
            save_path: 저장 경로 (None이면 자동 생성)
        """

        rcp_no = pdf_info.get("rcpNo")
        dcm_no = pdf_info.get("dcmNo")
        
        headers = self.headers
        headers['Referer'] = self.main_url.format(rcp_no=rcp_no)

        """
        https://dart.fss.or.kr/pdf/download/main.do?rcp_no=20251030800076&dcm_no=10857989
        https://dart.fss.or.kr/pdf/download/zip.do?rcp_no=20251030800076&dcm_no=10857989
        https://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20251201000615&dcm_no=10905849
        """
        """
        https://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20251030800076&dcm_no=10857989
        """

        download_urls = ['pdf.do', 'zip.do']

        for download_url in download_urls:
            url = self.pdf_url.format(rcp_no=rcp_no, dcm_no=dcm_no).replace('pdf.do',download_url)
            print(f"Downloading file: {url}")
            save_path = self.__download_file(url, headers, save_path)
            if save_path:
                break

        return save_path

    def __download_file(self, url, headers, save_path):
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"다운로드 실패: {response.status_code}")
            return None
        
        response_headers = response.headers
        content_length = response_headers.get("Content-Length")
        #print(response_headers, content_length)

        if content_length == '0':
            print(f"다운로드 형색 변경 필요. {response.status_code}")
            return None
        
        if save_path is None:
            content_disposition = response_headers.get("Content-Disposition")
            # filename="..." 또는 filename*=UTF-8''... 패턴 찾기
            match = re.search(r"filename\*?=['\"]?(?:UTF-8'')?([^'\";\n]+)", content_disposition)
            if match:
                filename = match.group(1)

                # URL 인코딩된 경우
                if '%' in filename:
                    filename = unquote(filename)
                
                # 깨진 인코딩 복원
                save_path = self.__encoding(filename)
            else:
                save_path = f"dart_{rcp_no}_{dcm_no}.pdf"
    
        with open(save_path, 'wb') as f:
            f.write(response.content)
    
        print(f"PDF 저장 완료: {save_path}")
        return save_path

    def __encoding(self, text):
        """깨진 한글 인코딩 복원"""
        # EUC-KR로 인코딩된 문자열이 Latin-1(ISO-8859-1)로 잘못 해석된 경우
        try:
            enc_text = text.encode('latin-1').decode('euc-kr')
            return enc_text
        except:
            pass
        
        # CP949로 시도
        try:
            enc_text = text.encode('latin-1').decode('cp949')
            return enc_text
        except:
            pass
        
        return text
    
    def _parse_download_page(self, pdf_info: dict) -> dict[str, str]:
        """
        다운로드 페이지에서 첨부파일 목록을 파싱합니다.
        
        Args:
            pdf_info: Dictionary (접수번호, 문서번호)
            
        Returns:
            파일명과 다운로드 URL의 딕셔너리
        """
        rcp_no = pdf_info.get("rcpNo")
        dcm_no = pdf_info.get("dcmNo")
        
        #download_url = f"http://dart.fss.or.kr/pdf/download/main.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
        download_url = self.pdf_main_url.format(rcp_no=rcp_no,dcm_no=dcm_no)
        
        response = requests.get(download_url, headers=self.headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "lxml")
        table = soup.find("table")
        
        if not isinstance(table, Tag):
            return {}
        
        return self._extract_files_from_table(table)
    
    def _extract_files_from_table(self, table: Tag) -> dict[str, str]:
        """테이블에서 파일 정보를 추출합니다."""
        files: dict[str, str] = {}
        
        tbody = table.find("tbody")
        if not isinstance(tbody, Tag):
            return files
        
        for row in tbody.find_all("tr"):
            if not isinstance(row, Tag):
                continue
            
            file_info = self._extract_file_from_row(row)
            if file_info:
                filename, url = file_info
                files[filename] = url
        
        return files
    
    def _extract_file_from_row(self, row: Tag) -> tuple[str, str] | None:
        """테이블 행에서 파일 정보를 추출합니다."""
        cells = row.find_all("td")
        if len(cells) < 2:
            return None
        
        filename_cell, link_cell = cells[0], cells[1]
        if not (isinstance(filename_cell, Tag) and isinstance(link_cell, Tag)):
            return None
        
        anchor = link_cell.find("a")
        if not isinstance(anchor, Tag):
            return None
        
        href = anchor.get("href")
        if href is None:
            return None
        
        filename = filename_cell.get_text(strip=True)
        url = f"http://dart.fss.or.kr{href}"
        
        return filename, url

#### DartDocumentParser 테스트

In [ ]:
#rcept_no = "20251201000783"
rcept_no = "20251030800076"
parser = DartDocumentParser()
list = parser.fetch(rcept_no)
list

In [56]:
class DartAPIParser:
    """
    OpenDART API 파싱 클래스
    
    공시정보: Public Disclosure, https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001
    정기보고서 주요정보: Key Information in Periodic Reports, https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS002
    정기보고서 재무정보: Financial Information in Periodic Reports, https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS003
    지분공시 종합정보: Comprehensive Share Ownership Information, https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS004
    주요사항보고서 주요정보: Key Information in Reports on Material Facts, https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS005
    증권신고서 주요정보: Key Information in Registration Statements, https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS006
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/120.0.0.0 Safari/537.36',
        #'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }

    list_url = "https://opendart.fss.or.kr/api/list.json"
    company_url = "https://opendart.fss.or.kr/api/company.json"
    document_url = "https://opendart.fss.or.kr/api/document.xml"
    corp_code_url = "https://opendart.fss.or.kr/api/corpCode.xml"

    finance_urls = {
        "단일회사 주요계정": "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json",
        "다중회사 주요계정": "https://opendart.fss.or.kr/api/fnlttMultiAcnt.json",
        "재무제표 원본파일(XBRL)": "https://opendart.fss.or.kr/api/fnlttXbrl.xml",
        "단일회사 전체 재무제표": "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json",
        "XBRL택사노미재무제표양식": "https://opendart.fss.or.kr/api/xbrlTaxonomy.json",
        "단일회사 주요 재무지표": "https://opendart.fss.or.kr/api/fnlttSinglIndx.json",
        "다중회사 주요 재무지표": "https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json"
    }

    def __init__(self, api_key):
        self.api_key = api_key

    def list(self, code, start: str | None = None, end: str | None = None):
        """
        OpenDart 공시정보 - 공시검색 

        Args:
            code (str): 기업 고유번호 (주식 코드(stock_code) 및 DART 고유번호 모두 가능(corp_code))
            start (str): 시작일 (YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD, YYYYMMDD)
            end (end): 종료일 (YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD, YYYYMMDD)
        Returns:
            pd.DataFrame: 공시 목록
        """
        start = datetime.now().strftime("%Y%m%d") if not start else self._dateformat(start)
        end = datetime.now().strftime("%Y%m%d") if not end else self._dateformat(end)
        
        params = {
            "crtfc_key": self.api_key,
            "corp_code": code,
            "bgn_de": start,
            "end_de": end,
            "corp_cls": "Y",
            "page_no": 1,
            "page_count": 100
        }

        all_data = []  # 전체 데이터 저장
        page = 1
        
        while True:
            params['page_no'] = page
            
            response = requests.get(self.list_url, params=params, headers=self.headers)
            response.encoding = 'utf-8'
            
            json_data = response.json()
            #print(json_data)
            
            status = json_data.get("status")

            # 에러 체크
            if status != "000":
                print(f"Error: {json_data.get('message')}")
                break

            # 데이터 추가
            data_list = json_data.get("list", [])
            all_data.extend(data_list)
            
            # 페이지 정보
            page_no = json_data.get("page_no", 1)
            total_page = json_data.get("total_page", 1)
            total_count = json_data.get("total_count", 0)

            print(f"페이지 {page}/{total_page} 완료 (총 {total_count}건)")

            # 마지막 페이지면 종료
            if page >= total_page:
                break

            page += 1

        # DataFrame 변환
        df = pd.DataFrame(all_data)
        return df

    def company(self, code):
        """
        OpenDart 공시정보 - 기업개황 (기업 정보 조회)
        corp_code, stock_code으로 조회가 가능하지만 기업명으로는 조회되지 않는다.

        Args:
            code (str): 기업 고유번호  (주식 코드(stock_code) 및 DART 고유번호 모두 가능(corp_code))
        Returns:
            Dict: 기업개황
        """
        
        params = {
            "crtfc_key": self.api_key,
            "corp_code": code
        }

        response = requests.get(self.company_url, params=params, headers=self.headers)
        response.encoding = 'utf-8'
        
        json_data = response.json()
        print(json_data)
            
        status = json_data.get("status")

        # 에러 체크
        if status != "000":
            print(f"Error: {json_data.get('message')}")
            return {}

        self.corp_code = json_data.get("corp_code")
        self.corp_name = json_data.get("corp_name")
        self.stock_code = json_data.get("stock_code")

        return json_data

    def document(self, rcept_no, save_path: str | None = None):
        """
        OpenDart 공시정보 - 공시서류원본파일
        https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019003

        Args:
            rcept_no (str): 접수번호
            save_path (str): 파일 저장 경로
        Returns:
            pd.DataFrame: 기업개황
        """
        
        params = {
            "crtfc_key": self.api_key,
            "rcept_no": rcept_no
        }

        response = requests.get(self.document_url, params=params, headers=self.headers)
        response.encoding = 'utf-8'
        response_headers = response.headers
        print(response_headers)

        if response.status_code != 200:
            print(f"다운로드 실패: {response.status_code}")
            return None

        content_type = response_headers.get("Content-Type")
        if "application/xml" in content_type:
            text_data = response.text
            print(text_data)
            return None

        # 바이너리 데이터 
        binary_data = response.content

        #content_length = response_headers.get("Content-Length")
        #print(response_headers, content_length)
        
        if save_path is None:
            content_disposition = response_headers.get("Content-Disposition", "")
            # filename="..." 또는 filename*=UTF-8''... 패턴 찾기
            match = re.search(r"filename\*?=['\"]?(?:UTF-8'')?([^'\";\n]+)", content_disposition)
            if match:
                filename = match.group(1)

                # URL 인코딩된 경우
                if '%' in filename:
                    filename = unquote(filename)
                
                # 깨진 인코딩 복원
                save_path = self.__encoding(filename)
                
                # .zip 확장자 제거하여 폴더명으로 사용
                save_path = filename.replace('.zip', '')
            else:
                save_path = f"dart_{rcept_no}"

        #self._save_zip(binary_data, save_path)

        result = {
            'rcept_no': rcept_no,
            'files': [],
            'xml_data': []
        }

        # ZIP 압축 해제 (메모리에서)
        with zipfile.ZipFile(io.BytesIO(binary_data)) as zf:
            file_list = zf.namelist()
            print(f"압축 파일 내 {len(file_list)}개 파일:")
            
            for fname in file_list:
                # 파일명 인코딩 수정
                enc_name = self.__encoding(fname)
                
                print(f"  - {enc_name}")
                result['files'].append(enc_name)
                
                # XML 파일만 파싱
                if fname.endswith('.xml'):
                    content = zf.read(fname)
                    parsed = self._parse_xml(content, enc_name)
                    result['xml_data'].append(parsed)
        
        print(f"\n총 {len(result['xml_data'])}개 XML 파일 파싱 완료")
        return result

    def corp_code(self, save_path: str | None = None):
        """
        OpenDart 공시정보 - 고유번호
        https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018

        Args:
            save_path (str): 파일 저장 경로
        Returns:
            Dict: 기업 고유번호 목록
        """
        
        params = {
            "crtfc_key": self.api_key
        }

        response = requests.get(self.corp_code_url, params=params, headers=self.headers)
        response.encoding = 'utf-8'
        response_headers = response.headers
        print(response_headers)

        if response.status_code != 200:
            print(f"다운로드 실패: {response.status_code}")
            return None

        content_type = response_headers.get("Content-Type")
        if "application/xml" in content_type:
            text_data = response.text
            print(text_data)
            return None

        # 바이너리 데이터 
        binary_data = response.content

        #content_length = response_headers.get("Content-Length")
        #print(response_headers, content_length)
        
        if save_path is None:
            content_disposition = response_headers.get("Content-Disposition", "")
            # filename="..." 또는 filename*=UTF-8''... 패턴 찾기
            match = re.search(r"filename\*?=['\"]?(?:UTF-8'')?([^'\";\n]+)", content_disposition)
            if match:
                filename = match.group(1)

                # URL 인코딩된 경우
                if '%' in filename:
                    filename = unquote(filename)
                
                # 깨진 인코딩 복원
                save_path = self.__encoding(filename)
                
                # .zip 확장자 제거하여 폴더명으로 사용
                save_path = filename.replace('.zip', '')
            else:
                save_path = f"dart_{rcept_no}"

        #self._save_zip(binary_data, save_path)

        result = {
            'files': [],
            'xml_data': []
        }

        # ZIP 압축 해제 (메모리에서)
        with zipfile.ZipFile(io.BytesIO(binary_data)) as zf:
            file_list = zf.namelist()
            print(f"압축 파일 내 {len(file_list)}개 파일:")
            
            for fname in file_list:
                # 파일명 인코딩 수정
                enc_name = self.__encoding(fname)
                
                print(f"  - {enc_name}")
                result['files'].append(enc_name)
                
                # XML 파일만 파싱
                if fname.endswith('.xml'):
                    content = zf.read(fname)
                    parsed = self._parse_xml(content, enc_name)
                    result['xml_data'].append(parsed)
        
        print(f"\n총 {len(result['xml_data'])}개 XML 파일 파싱 완료")
        return result

    def finance(self, code: str, year: str, report_code: str = '11011'):
        """
        OpenDart 정기보고서 재무정보 - 단일회사 주요 재무지표
        corp_code으로만 조회가 가능, stock_code, 기업명으로는 조회되지 않는다.
        https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2022001

        보고서 코드:
        1분기보고서 : 11013
        반기보고서 : 11012
        3분기보고서 : 11014
        사업보고서 : 11011

        지표분류코드:
        수익성지표 : M210000
        안정성지표 : M220000
        성장성지표 : M230000
        활동성지표 : M240000

        재무제표구분: (※재무제표구분 참조)

        개별/연결구분: (단일회사 전체 재무제표)
        OFS:재무제표
        CFS:연결재무제표

        Args:
            code (str): 기업 고유번호  (주식 코드(stock_code) 및 DART 고유번호 모두 가능(corp_code))
            year (str): 사업연도
            report_code (str): 보고서 코드, 기본값: 11011
        Returns:
            Dict: 기업개황
        """

        #corp_code,bsns_year,stacnt_code,idx_cl_code
        
        params = {
            "crtfc_key": self.api_key,
            "corp_code": code,
            "bsns_year": year,
            "reprt_code": report_code,
            "idx_cl_code": "M210000",
            "sj_div": "BS1",
            "fs_div": "OFS"
        }

        #url = self.finance_urls.get("단일회사 주요계정", "")
        #url = self.finance_urls.get("다중회사 주요계정", "")
        #url = self.finance_urls.get("단일회사 전체 재무제표", "")
        url = self.finance_urls.get("XBRL택사노미재무제표양식", "")
        #url = self.finance_urls.get("단일회사 주요 재무지표", "")
        #url = self.finance_urls.get("다중회사 주요 재무지표", "")

        response = requests.get(url, params=params, headers=self.headers)
        response.encoding = 'utf-8'
        
        json_data = response.json()
        print(json_data)
            
        status = json_data.get("status")

        # 에러 체크
        if status != "000":
            print(f"Error: {json_data.get('message')}")
            return {}

        self.corp_code = json_data.get("corp_code")
        self.corp_name = json_data.get("corp_name")
        self.stock_code = json_data.get("stock_code")

        return json_data

    def _parse_xml(self, xml_content, filename=None):
        """
        XML 내용 파싱
        Args:
            xml_content (bytes): XML 바이너리 내용
            filename (str): 파일명
        Returns:
            dict: 파싱된 데이터
        """
        from bs4 import BeautifulSoup
        
        # bytes를 str로 변환
        xml_str = None
        for encoding in ['utf-8', 'euc-kr', 'cp949']:
            try:
                xml_str = xml_content.decode(encoding)
                break
            except:
                continue
        
        if xml_str is None:
            return {'filename': filename, 'error': '인코딩 실패'}
        
        soup = BeautifulSoup(xml_str, 'lxml-xml')
        
        result = {
            'filename': filename,
            'title': None,
            'tables': [],
            'text_content': None,
            'list': []
        }
        
        # 제목 추출
        title_tag = soup.find('TITLE') or soup.find('title')
        if title_tag:
            result['title'] = title_tag.get_text(strip=True)
        
        # 테이블 추출
        tables = soup.find_all('TABLE') or soup.find_all('table')
        for idx, table in enumerate(tables):
            rows = []
            for tr in table.find_all(['TR', 'tr']):
                row = []
                for cell in tr.find_all(['TD', 'TH', 'td', 'th']):
                    text = cell.get_text(separator=' ', strip=True)
                    text = re.sub(r'\s+', ' ', text)
                    row.append(text)
                if row:
                    rows.append(row)
            
            if rows:
                # DataFrame으로 변환
                df = self._rows_to_dataframe(rows)
                
                result['tables'].append({
                    'index': idx,
                    'data': rows,
                    'dataframe': df
                })
        
        # 텍스트 내용 추출
        body = soup.find('BODY') or soup.find('body')
        if body:
            result['text_content'] = body.get_text(separator='\n', strip=True)

        # list 태그들 추출
        result_list = soup.find('result')
        if result_list:
            data = []
            for item in result_list.find_all('list'):
                # XML을 JSON 리스트로 변환
                row = {child.name: child.get_text(strip=True) for child in item.children if child.name}
                data.append(row)
            result['list'] = data
        
        return result

    def _rows_to_dataframe(self, rows):
        """
        2D 리스트를 DataFrame으로 안전하게 변환
        """
        if not rows:
            return pd.DataFrame()
        
        if len(rows) == 1:
            return pd.DataFrame(rows)
        
        # 최대 컬럼 수 계산
        max_cols = max(len(row) for row in rows)
        
        # 모든 행의 길이를 최대 컬럼 수에 맞춤
        normalized = []
        for row in rows:
            if len(row) < max_cols:
                row = row + [''] * (max_cols - len(row))
            normalized.append(row)
        
        # 첫 행을 헤더로 사용
        header = normalized[0]
        data = normalized[1:]
        
        # 빈 헤더 처리
        header = [f"col_{i}" if not h else h for i, h in enumerate(header)]
        
        # 헤더 중복 처리
        seen = {}
        unique_header = []
        for col in header:
            if col in seen:
                seen[col] += 1
                unique_header.append(f"{col}_{seen[col]}")
            else:
                seen[col] = 0
                unique_header.append(col)
        
        return pd.DataFrame(data, columns=unique_header)

    def _dateformat(self, date_str):
        """다양한 날짜 형식을 YYYYMMDD로 변환"""
        # 구분자(-, /, .) 제거
        return re.sub(r'[-/.]', '', date_str)

    def _save_zip(self, binary_data, save_path):
        # ZIP 압축 해제
        with zipfile.ZipFile(io.BytesIO(binary_data)) as zf:
            # 파일 목록 출력
            file_list = zf.namelist()
            print(f"압축 파일 내 {len(file_list)}개 파일:")
            for fname in file_list:
                print(f"  - {fname}")
            
            # 전체 압축 해제
            zf.extractall(save_path)
        
        print(f"압축 해제 완료: {save_path}")

    def __encoding(self, text):
        """깨진 한글 인코딩 복원"""
        # EUC-KR로 인코딩된 문자열이 Latin-1(ISO-8859-1)로 잘못 해석된 경우
        try:
            enc_text = text.encode('latin-1').decode('euc-kr')
            return enc_text
        except:
            pass
        
        # CP949로 시도
        try:
            enc_text = text.encode('latin-1').decode('cp949')
            return enc_text
        except:
            pass
        
        return text

In [57]:
opendart_api = DartAPIParser("fd664865257f1a3073b654f9185de11a708f726c")
#df = opendart_api.list("005930", start="2025.10.01", end="2025.10.31")
#df = opendart_api.list("005930")
#df
#opendart_api.company("005930")
#opendart_api.company("00126380")
#opendart_api.company("심성전자")

#opendart_api.document("20251204000046")
#'Content-Type': 'application/x-msdownload;charset=UTF-8'
#opendart_api.document("20251299999999")
#'Content-Type': 'application/xml;charset=UTF-8'

#opendart_api.corp_code()
#opendart_api.finance("005930", "2024")
#opendart_api.finance("심성전자", "2024")
#{'status': '013', 'message': '조회된 데이타가 없습니다.'}
#opendart_api.finance("00126380", "2025")
#{'status': '013', 'message': '조회된 데이타가 없습니다.'}
opendart_api.finance("00126380", "2024")

{'status': '000', 'message': '정상', 'list': [{'sj_div': 'BS1', 'bsns_de': '20180701', 'account_id': 'ifrs_StatementOfFinancialPositionAbstract', 'account_nm': 'StatementOfFinancialPositionAbstract', 'label_kor': '재무상태표 [abstract]', 'label_eng': 'Statement of financial position [abstract]', 'ifrs_ref': ' '}, {'sj_div': 'BS1', 'bsns_de': '20180701', 'account_id': 'ifrs_AssetsAbstract', 'account_nm': 'AssetsAbstract', 'label_kor': '자산 [abstract]', 'label_eng': 'Assets [abstract]', 'ifrs_ref': ' '}, {'sj_div': 'BS1', 'bsns_de': '20180701', 'account_id': 'ifrs_CurrentAssets', 'account_nm': 'CurrentAssets', 'label_kor': '유동자산', 'label_eng': 'Current assets', 'data_tp': 'X', 'ifrs_ref': 'K-IFRS 1001 문단 60'}, {'sj_div': 'BS1', 'bsns_de': '20180701', 'account_id': 'ifrs_CashAndCashEquivalents', 'account_nm': 'CashAndCashEquivalents', 'label_kor': '현금및현금성자산', 'label_eng': 'Cash and cash equivalents', 'data_tp': 'X', 'ifrs_ref': 'K-IFRS 1001 문단 54 (9),K-IFRS 1007 문단 45'}, {'sj_div': 'BS1', 'bsns_d

{'status': '000',
 'message': '정상',
 'list': [{'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_StatementOfFinancialPositionAbstract',
   'account_nm': 'StatementOfFinancialPositionAbstract',
   'label_kor': '재무상태표 [abstract]',
   'label_eng': 'Statement of financial position [abstract]',
   'ifrs_ref': ' '},
  {'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_AssetsAbstract',
   'account_nm': 'AssetsAbstract',
   'label_kor': '자산 [abstract]',
   'label_eng': 'Assets [abstract]',
   'ifrs_ref': ' '},
  {'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_CurrentAssets',
   'account_nm': 'CurrentAssets',
   'label_kor': '유동자산',
   'label_eng': 'Current assets',
   'data_tp': 'X',
   'ifrs_ref': 'K-IFRS 1001 문단 60'},
  {'sj_div': 'BS1',
   'bsns_de': '20180701',
   'account_id': 'ifrs_CashAndCashEquivalents',
   'account_nm': 'CashAndCashEquivalents',
   'label_kor': '현금및현금성자산',
   'label_eng': 'Cash and cash equivalents',
   'data_tp': 'X'

#### OpenDartCrawler

In [ ]:
class OpenDartCrawler:
    """DART 공시 문서 크롤러.
    
    기업의 공시 문서를 DART에서 크롤링하여 GCS에 업로드합니다.
    """
    
    # 제외할 공시 유형
    EXCLUDED_REPORT_TYPES = frozenset({"기업설명회(IR)개최(안내공시)"})
    request_delay_seconds = 3

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/120.0.0.0 Safari/537.36'
    }
    
    def __init__(self, code: str = "005930"):
        """크롤러를 초기화합니다.
        
        Args:
            code: 기업 코드 (기본값: 삼성전자)
        """
        self.config = DartConfig.from_env()
        self.code = code
        #self.dart = OpenDartReader(self.config.api_key)
        self.dart = OpenDartReader("fd664865257f1a3073b654f9185de11a708f726c")
    
    def fetch(
        self,
        code: str | None = None,
        start_date: str = "2025-01-01",
        end_date: str = "2025-08-19",
        count: int = 5,
    ) -> None:
        """기업의 기본 공시 문서를 크롤링합니다.
        
        Args:
            code: 기업 코드 (None이면 초기화 시 설정된 값 사용)
            start_date: 검색 시작일 (YYYY-MM-DD)
            end_date: 검색 종료일 (YYYY-MM-DD)
            count: 크롤링할 최대 문서 수
        """
        if code:
            self.code = code
        
        company_name = companydict.get_company(self.code)
        folder_path = f"OpenDart/{company_name}/"
        
        #reports_to_process = self._get_reports_to_process(
        #    start_date, end_date, folder_path, count
        #)
        list = self._fetch_list(
            code, start_date, end_date
        )
        
        #for rcept_no, flr_nm in reports_to_process:
        #    self._process_single_report(rcept_no, flr_nm, folder_path)
        #    time.sleep(self.request_delay_seconds)

    def _fetch_list(
        self,
        code: str,
        start_date: str,
        end_date: str
    ) -> pd.DataFrame:
        df = self.dart.list(code, start=start_date, end=end_date)
        df["report_nm"] = df["report_nm"].str.strip()

        # 이미 다운로드된 파일 확인
        existing_files = self.gcs_manager.list_files(folder_name=folder_path)
        print(existing_files)

        return df
    
    def _get_reports_to_process(
        self,
        start_date: str,
        end_date: str,
        folder_path: str,
        count: int,
    ) -> List[Tuple[str, str]]:
        """
        처리할 공시 목록을 가져옵니다.

        Args:
            rcept_no (str):

        Returns:
            Dictionary
        """
        df = self.dart.list(self.code, start=start_date, end=end_date)
        #print(df)
        df["report_nm"] = df["report_nm"].str.strip()

        
        # 이미 다운로드된 파일 확인
        existing_files = self.gcs_manager.list_files(folder_name=folder_path)
        existing_rcept_nos = {
            f.split("_")[-1].replace(".pdf", "") for f in existing_files
        }
        
        # 필터링 조건 적용
        is_new = ~df["rcept_no"].isin(existing_rcept_nos)
        is_not_excluded = ~df["report_nm"].isin(self.EXCLUDED_REPORT_TYPES)
        
        filtered_df = df[is_new & is_not_excluded].head(count)
        
        return list(zip(filtered_df["rcept_no"], filtered_df["flr_nm"]))
    
    def _process_single_report(
        self,
        rcept_no: str,
        flr_nm: str,
        folder_path: str,
    ) -> None:
        """
        단일 공시를 처리합니다.

        Args:
            rcept_no (str):

        Returns:
            Dictionary
        """
        files = self._get_attachment_files(rcept_no)
        
        if not files:
            print(f"첨부파일을 찾지 못했습니다 (접수번호: {rcept_no})")
            return
        
        for filename, url in files.items():
            self._download_and_upload_file(filename, url, rcept_no, folder_path)
    
    def _get_attachment_files(self, rcept_no: str) -> Dict[str, str]:
        """
        첨부파일 목록을 가져옵니다.

        Args:
            rcept_no (str):

        Returns:
            Dictionary
        """
        # OpenDartReader의 내장 메서드 우선 시도
        files = self.dart.attach_files(rcept_no)
        
        if files:
            return files
        
        # 실패 시 직접 파싱 시도
        try:
            return DartDocumentParser.fetch(rcept_no)
        except requests.RequestException as e:
            print(f"네트워크 오류 (접수번호: {rcept_no}): {e}")
        except Exception as e:
            print(f"파싱 오류 (접수번호: {rcept_no}): {e}")
        
        return {}
    
    def _download_and_upload_file(
        self,
        filename: str,
        url: str,
        rcept_no: str,
        folder_path: str,
    ) -> None:
        """
        파일을 다운로드하고 GCS에 업로드합니다.

        Args:
            rcept_no (str):
        """
        # 파일명 정규화
        if not filename:
            filename = url.split("/")[-1]
        
        # HTML 파일 제외
        if filename.endswith(".html"):
            return
        
        # 접수번호를 파일명에 추가
        destination_name = filename.replace(".pdf", f"_{rcept_no}.pdf")
        
        try:
            response = requests.get(url, stream=True, headers=self.headers, timeout=60)
            response.raise_for_status()
            
            self.gcs_manager.upload_file(
                source_file=response.content,
                destination_blob_name=folder_path + destination_name,
            )
        except requests.RequestException as e:
            print(f"파일 다운로드/업로드 실패 ({filename}): {e}")

#### OpenDartCrawler 테스트

In [ ]:
crawler = OpenDartCrawler()
data = crawler.fetch(code="005930")